In [1]:
## Each Space is a cell in a juptyr notebook ("QuickThinking" in fastai_workbench) that runs on Gradient Paperspace
from greek_normalisation.utils import (nfd, nfc, nfkc, convert_to_2019)
from greek_accentuation.characters import base
from cltk.lemmatize.grc import GreekBackoffLemmatizer
from alphabet import filter_non_greek, filter_non_greek_with_punctuation
from alphabet import PUNCTUATION
from cltk.stops.grc import STOPS
from greek_swadesh import SWADESH
from chrysostom_junk_words import JUNK_WORDS
import numpy as np

#greek_text = 'ὁ αὐτὸς μῆνιν ἄειδε θεὰ Πηληϊάδεω Ἀχιλῆος; \nκαὶ κατέβην χθὲς εἰς Πειραιᾶ μετὰ Γλαύκωνος τοῦ Ἀρίστωνος. Τιμταμ;'
file = open('/notebooks/storage/greek_text/ALL_REAL_JOHN_FOR_BAG_OF_WORDS.TXT')
greek_text = file.read().lower()
file.close()
all_text = greek_text.split('.')
print("total sentences:")
total_sentences = len(all_text)
print(total_sentences)
#no_of_sentences = total_sentences * 0.25
#no_of_sentences = total_sentences * 0.50
#no_of_sentences = total_sentences * 0.75
no_of_sentences = total_sentences * 1
selection = all_text[0:int(no_of_sentences)] #grab only the first XXXX sentences for testing
print("selected sentences:")
print(len(selection))
percent_selected = (len(selection) / len(all_text) * 100)
print("percent selected for processeing:")
print(percent_selected)
greek_text = '.'.join(selection)
original_text = greek_text
lemmatizer = GreekBackoffLemmatizer()

def strip_diacriticals(greek_text):
    return ''.join([base(char) for char in greek_text])

def normalize_greek(greek_text):
    greek_text = nfc(greek_text) #normalize pre-composed unicode characters to be single characters
    greek_text = convert_to_2019(greek_text).lower() #normalize apostrophes and lowercase
    greek_text = filter_non_greek_with_punctuation(greek_text) #strip everything but greek and basic punctuation
    greek_text = greek_text.replace(';','.').replace('·','.') #replace all punctuation except ","s with "."s
    return greek_text

def unhelpful_words():
    stops = sorted(normalize_greek(' '.join(STOPS)).split(' '))
    swadesh = sorted(normalize_greek(' '.join(SWADESH)).split(' '))
    empty = ['', ' ', '\n']
    common = ['ἄνευ','εισ','ὡσανεὶ','θ','ταῦτ','πρό','δήποτε', 'δεύτερος', 'αἰών','τοσοῦτος', 'ποτέ','ἴσως', 'ἐκεῖνος','ἐπεί','ἀνά','εἴτε','φησὶ,','ἰδοὺ', 'εἶπε,', 'πῶς,', 'τοσοῦτον,', 'ὅσος','μηδείς', 'μηδέ', 'μήν','γὰρ,', 'εἶπον','ἐστὶν,','ἐστὶν', 'ὅτε','ποῖος','μοι,','ἐρῶ','τοῦτο,','ἀντί','πόσος','ἐστιν','εἶπεν','εὐθύς','ποῦ','ἔμπροσθεν','ἀπαιτέω','κἀν','διό' 'ποτέ' 'ἵημι' 'ἡμῶν,' 'σαφής' 'ὅπου', 'γίγνομαι', 'ἵημι', 'ἐντεῦθεν','ἕνεκα','σου','δοκέω','φησὶ', 'φημί','τίθημι','γὰρ','τοσοῦτος','ὅσπερ','πᾶς','μόνος','ὅταν','ἅπας','δύναμαι','καθά','τοσοῦτος','ἀρετή','ἡμέτερος','ἀεί','τουτέστι,','πλῆθος','αὐτὸν', 'εἶναι', 'ἦν', 'ἐστιν,' 'ἐπειδὴ', 'αὐτῷ', 'αὐτοὺς', 'τότε', 'τούτου', 'πάντες', 'αὕτη', 'ἦν', 'πάντας', 'ὅμως', 'φησιν', 'τουτέστι', 'ἔχειν', 'πλέον', 'ἑαυτοὺς', 'οὕτω', 'οὐχ', 'ὥσπερ', 'ἐστι', 'ἐστιν', 'μόνον', 'τοῦτό', 'ἔχει', 'ὑμᾶς', 'ἐστὶ', 'μέγα', 'γῆς', 'ποιεῖν', 'τοσοῦτον', 'ὅσον', 'μικρὸν', 'γοῦν', 'γέγονε', 'ἵνα', 'ἡμᾶς', 'ἡμῶν', 'σου', 'σε', 'εἶπεν', 'με', 'πᾶσαν', 'κατ', 'ἐκεῖνο', 'εἶπεν', 'αὐτῇ', 'γένοιτο', 'καθάπερ', 'ἔστι', 'ἐστιν', 'ὑμῶν', 'ἕτερον', 'οὗ', 'ἀεὶ',  'οἷς', 'λέγοντος', 'ἐμοὶ', 'νῦν', 'ταῦτα', 'πάντα', 'μόνον', 'ἡμῖν', 'τούτων', 'πάντων', 'οὐχὶ', 'ἐκείνων', 'μηδὲν', 'δὲ', 'τοῦτον', 'ἐστὶν',  'ἐκεῖνα', 'ἆρα', 'μετ', 'λέγων', 'α', 'ἄλλως', 'οὐδὲν', 'φησὶ', 'πάλιν', 'αὐτοῖς', 'εἶτα', 'πολλὰ', 'καλῶς', 'ἃ', 'τοιοῦτον', 'ἐκεῖνοι', 'εἶναι', 'χρὴ', 'μείζονα', 'τουτέστιν', 'αὑτοῦ', 'πάντοθεν', 'αἰτίαν', 'τοῦτο', 'τὰς', 'αἱ', 'μηδὲ', 'αὐτὴν', 'καίτοι', 'ἄλλων', 'ποιεῖ', 'γενέσθαι', 'ποτε', 'εἶπε', 'παντὸς', 'ἐκείνην', 'οἷον', 'ἀλλὰ', 'ἕκαστος', 'εἰπεῖν', 'φησὶν', 'δι', 'τοίνυν', 'ταύτην', 'πάλιν', 'ὧν', 'σφόδρα', 'σοι', 'εὐθέως', 'δῆλον', 'πολλὴν', 'ἕως', 'πόθεν', 'ἤδη', 'οὖν', 'εἰσιν', 'τὸ', 'οὐδέν', 'δεῖ', 'γὰρ', 'ὃ', 'λέγων', 'μου', 'τούτῳ', 'ἐπεὶ', 'πρῶτον', 'λέγει', 'οὐκοῦν', 'αὐτοὶ', 'οὗτοι', 'γ', 'αὐτόν', 'φησι', 'ταῖς', 'ὅπερ', 'ὁρᾷς', 'αὐτῆς', 'ὄντως', 'ἐκείνου', 'μήτε', 'μοι', 'διὸ', 'χωρὶς', 'ἅμα', 'εἰπέ', 'εἰπέ_μοι', 'ἔσται', 'ὁμοίως', 'ἐκείνῳ',  'αὐτοῦ', 'αὐτῶν', 'παρ', 'γίνεται', 'λέγω', 'οὐκέτι', 'τοιαῦτα', 'εἰκότως', 'αὐτοῦ', 'ἑτέρων', 'β', 'ἐκείνης', 'μὴν', 'πολλῆς', 'πολλάκις', 'ἁπλῶς', 'ἕνεκεν', 'αὐτὰ', 'ὑμῖν', 'τίνος', 'πάσης', 'φησι', 'μυρία', 'τίνος_ἕνεκεν', 'ἔχων', 'ἧς', 'νῦν', 'αὐτὸ', 'ἔστιν', 'φησίν', 'καθ', 'λέγει', 'μέχρι', 'πρὸ', 'πᾶν', 'τοῦτο', 'ἐκεῖνον', 'ἅπερ', 'πολλοὶ', 'ἄλλο', 'ἦσαν', 'ἔλεγεν', 'ἀμήν', 'μοι', 'μάλιστα', 'ταύτης', 'ἐστι', 'πανταχοῦ', 'φησί', 'λοιπὸν', 'εἰπεῖν', 'τινες', 'πολὺ', 'ἐστίν', 'ἅπαντα', 'ἀντὶ', 'οὐδαμοῦ', 'ἀλλήλους', 'μᾶλλον', 'κἂν', 'πολλῷ', 'εἴ', 'πολλῷ', 'μᾶλλον', 'νόμον', 'ἔνθα', 'ᾖ', 'τούτοις', 'ταχέως', 'εἶδες', 'ὅλως', 'οὐδέποτε']
    return sorted(list(set(np.concatenate((stops, swadesh, empty, common)))))

UNHELPFUL = unhelpful_words()
#print(UNHELPFUL)

def remove_words_unhelpful_for_lda(lemmata):
    helpful_words = [word for word in lemmata if word not in UNHELPFUL and word != '' and word != None]
    return helpful_words

def tokenize(sentence):
    sentence = sentence.replace(',','').split(' ')
    lemmatized_tuples = lemmatizer.lemmatize(sentence)
    lemmata = [tuple[1] for tuple in lemmatized_tuples]
    lemmata = remove_words_unhelpful_for_lda(lemmata)
    return lemmata

def make_lda_documents(greek_text):
    # LDA wants a List of Lists of words (= document), I will make each sentence a document
    print("chars in greek text:")
    print(len(greek_text))
    print("words in greek text:")
    print(len(greek_text.split(' ')))

    print("normalizing greek...")
    import datetime
    start_normalizing = datetime.datetime.now()
    print(start_normalizing)
    documents = normalize_greek(greek_text).split('.')
    print("...done.")
    done_normalizing = datetime.datetime.now()
    print(done_normalizing)

    print("tokenizing...")
    start_tokenizing = datetime.datetime.now()
    print(start_tokenizing)

    lda_documents = []
    sentences_per_document = 50 # doc ~ a paragraph / thought
    current_document = []

    for sentence in documents:
        if len(sentence) > 0:
            sentence = tokenize(sentence)
        if len(sentence) > 0:
            current_document.extend(sentence)
            if len(current_document) >= sentences_per_document:
                lda_documents.append(current_document)
                current_document = []

    print("...done.")
    done_tokenizing = datetime.datetime.now()
    print(done_tokenizing)

    print("number of lsa_documents:")
    print(len(lda_documents))

    print("peek at lsa_documents:")
    print(lda_documents[-3])


    return lda_documents

total sentences:
141400
selected sentences:
141400
percent selected for processeing:
100.0


In [ ]:
#Run Preprocessing
lda_documents = make_lda_documents(greek_text)

chars in greek text:
21049531
words in greek text:
3328304
normalizing greek...
2020-10-13 13:37:27.974069


In [ ]:
#Compute bigrams
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(lda_documents, min_count=20)
for idx in range(len(lda_documents)):
    for token in bigram[lda_documents[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            lda_documents[idx].append(token)

In [ ]:
#Remove rare and common tokens (Filter out words that occur less than 20 documents, or more than 50% of the documents)
from gensim.corpora import Dictionary
lda_dictionary = Dictionary(lda_documents)
lda_dictionary.filter_extremes(no_below=20, no_above=0.5)

In [ ]:
#Vectorize documents into Bag-of-words.
lda_corpus = [lda_dictionary.doc2bow(doc) for doc in lda_documents]
#Let’s see how many tokens and documents we have to train on.
print('Number of unique tokens: %d' % len(lda_dictionary))
print('Number of documents: %d' % len(lda_corpus))

In [ ]:
#5. Train LDA model.
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.models import LdaModel

# Set training parameters.
num_topics = 50 #15
chunksize = 1500
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = lda_dictionary[0]  # This is only to "load" the lda_dictionary.
id2word = lda_dictionary.id2token

model = LdaModel(
    corpus=lda_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

top_topics = model.top_topics(lda_corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)
from pprint import pprint
pprint(top_topics)